In [60]:
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error

In [61]:
data = pd.read_excel('data/data_ford_price.xlsx') 
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [62]:
data = data[['price','year', 'condition', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обор признаков с помощью метода RFE

In [63]:
from sklearn.feature_selection import RFE

In [64]:
random_state = 42
select = RFE(estimator=LinearRegression(),
             n_features_to_select=3,
             step=1)
select = select.fit(X_train, y_train)

select.get_feature_names_out()

array(['year', 'condition', 'cylinders'], dtype=object)

In [65]:
X_train_rfe = X_train[select.get_feature_names_out()]
X_test_rfe = X_test[select.get_feature_names_out()]

In [66]:
model = LinearRegression()

model.fit(X_train_rfe, y_train)
score_train = cross_val_score(estimator=model,
                              X=X_train_rfe,
                              y=y_train,
                              cv=5,
                              scoring='r2',
                              n_jobs=1)

print(f'Метрика R2 на тренировочных данных - {score_train.mean():.2f}')
y_pred = model.predict(X_test_rfe)
print(f'Метрика R2 на тестовых данных - {r2_score(y_test, y_pred):.2f}')
print(f'Метрика MAE на тестовых данных - {mean_absolute_error(y_test, y_pred):.0f}')

Метрика R2 на тренировочных данных - 0.52
Метрика R2 на тестовых данных - 0.59
Метрика MAE на тестовых данных - 5011


## Обор признаков с помощью метода SelectKBest

In [67]:
from sklearn.feature_selection import SelectKBest, f_regression

In [68]:
select = SelectKBest(f_regression, k=3)
select.fit(X_train, y_train)

select.get_feature_names_out()

array(['year', 'condition', 'odometer'], dtype=object)

In [69]:
X_train_kbest = X_train[select.get_feature_names_out()]
X_test_kbest = X_test[select.get_feature_names_out()]

In [70]:
model = LinearRegression()

model.fit(X_train_kbest, y_train)
score_train = cross_val_score(estimator=model,
                              X=X_train_kbest,
                              y=y_train,
                              cv=5,
                              scoring='r2',
                              n_jobs=1)

print(f'Метрика R2 на тренировочных данных - {score_train.mean():.2f}')
y_pred = model.predict(X_test_kbest)
print(f'Метрика R2 на тестовых данных - {r2_score(y_test, y_pred):.2f}')
print(f'Метрика MAE на тестовых данных - {mean_absolute_error(y_test, y_pred):.0f}')

Метрика R2 на тренировочных данных - 0.53
Метрика R2 на тестовых данных - 0.60
Метрика MAE на тестовых данных - 4802


## Вывод

Так как при отборе признаков не обязательно они совпадут, для сравнения качества моделей была выбрана метрика R2, так как она описывает какую долю данных смогла описать полученная модель и в качестве вспомогательной - MAE.
При выборе трех наиболее значимых признаков с помощью RFE и SelectKBest мы действительно получили разные массивы признаков и после обучения и предсказания модели линейной регрессии на каждом из них можно сделать вывод, что SelectKBet в данном случае показал более хороший результат. R2 метрика на тестовых данных имеет значение 0,6, против 0,59 у модели, построенной на признаках, полученных с помощью RFE. Аналогичные показатели и на тренировочных данных - 0,53 и 0,52 соответственно. Также по вспомогательное метрике - среднему абсолютному отклонению видно что вторая модель в среднем ошибается на 4802, а первая на 5011.